# Initial AI Agent PPO_render (not working)

In [1]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [2]:
%cd "J:\git\TensorFlowPSX\Py"

from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import numpy as np
import gymnasium
from time import sleep

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 2000
my_config["act_buf_len"] = 4
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'img_hist_len': 3,
  'discreteAccel' : False,
  'accelAndBrake' : False,
  'discSteer' : False,
  'contAccelOnly' : True,
  'discAccelOnly' : False,
  'modelMode': 4,
  'agent' : 'PPO',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 4, # 1 is High Speed Ring x MR2, 2 is 0-400m x MR2, 3 is 0-400 Supra disc, 4 is 0-400 Supra cont, 
}

j:\git\TensorFlowPSX\Py


In [3]:
def env_creator(env_config):
    env = gymnasium.make("real-time-gym-v1", config=env_config)
    return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [4]:
import ray
ray.shutdown()
ray.init()

2023-10-04 21:29:33,003	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.13
Ray version:,2.6.3
Dashboard:,http://127.0.0.1:8265


In [5]:
from ray.rllib.algorithms.ppo import PPOConfig

algo = (
    PPOConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        enable_connectors=True,
        batch_mode="truncate_episodes",
        )
    .framework(
        framework="torch",
        )
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=True,
        )
    .training(
        train_batch_size=128,
    )
    .evaluation(
        # Evaluate once per training iteration.
        evaluation_interval=5,
        # Run evaluation on (at least) two episodes
        evaluation_duration=1,
        # ... using one evaluation worker (setting this to 0 will cause
        # evaluation to run on the local evaluation worker, blocking
        # training until evaluation is done).
        evaluation_num_workers=0,
        # Special evaluation config. Keys specified here will override
        # the same keys in the main config, but only for evaluation.
        evaluation_config=PPOConfig.overrides(
            # Render the env while evaluating.
            # Note that this will always only render the 1st RolloutWorker's
            # env and only the 1st sub-env in a vectorized env.
            render_env=True,
        )
    )
    .build()
)


#SACConfig.framework("")

2023-10-04 21:29:35,628	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-10-04 21:29:36,243	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-10-04 21:29:36,244	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `c

(RolloutWorker pid=13976) GT Real Time instantiated
(RolloutWorker pid=13976) GT AI Server instantiated for rtgym
(RolloutWorker pid=13976) starting up on localhost port 9999
(RolloutWorker pid=13976) Waiting for a connection
(RolloutWorker pid=13976) Connection from ('127.0.0.1', 55529)
(RolloutWorker pid=13976) control 6 model mode 4
GT Real Time instantiated
GT AI Server instantiated for rtgym


OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted

In [ ]:
N = 1000

for n in range(N):
    result = algo.train()
    print("Loop: ", n)
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
        
algo.save()
